## MetaPhlAn4 (vJun23) and Kraken2Uniq Taxa Concordance 

### Quantify the overlap between taxa based on NCBI ID's between the reference databases used by MetaPhlAn4 database vJun23 and custom Kraken2Uniq database, and concordance between the resulting taxa identified in ILO dataset

#### Author: Sarah Bald, Date: May 13, 2024
#### Edited: Sarah Bald, Date: November 12, 2024

In [1]:
import pickle
import bz2
import pandas as pd
import os

### Source Data

In [2]:
#source data paths, only change these when things get updated

#metaphlan ilo data - source listed and csv version produced from script save_rds_as_csv.R
mph_ilo_source_file = '/restricted/projectnb/uh2-sebas/data/metagenomics/ILO_combined_cohort/processed_data/data_library/metaphlan4_renorm_ILO_phyloseq.09.30.2024.rds'
mph_ilo_file = '/restricted/projectnb/uh2-sebas/analysis/metagenomics/taxa_concordance/metaphlan4_renorm_ILO_phyloseq.09.30.2024.csv'

#metaphlan han data
mph_xu_source_file = '/restricted/projectnb/uh2-sebas/data/metagenomics/external_cohorts/Han_Chinese_Centenarian/processed_data/data_library/metaphlan4_renorm_chinese_phyloseq.10.23.2024.rds'
mph_xu_file = '/restricted/projectnb/uh2-sebas/analysis/metagenomics/taxa_concordance/metaphlan4_renorm_xu_phyloseq.10.23.2024.csv'

#metaphlan vJun23 database
mph_db_file = '/restricted/projectnb/uh2-sebas/analysis/metagenomics/taxa_concordance/mpa_vJun23_CHOCOPhlAnSGB_202403.pkl'

#bracken ilo data - source listed and csv version produced from script save_rds_as_csv.R
bracken_ilo_source_file = '/restricted/projectnb/uh2-sebas/data/metagenomics/ILO_combined_cohort/processed_data/data_library/bracken_renorm_ILO_phyloseq.R1.K3.10.09.2024.rds'
bracken_ilo_file = '/restricted/projectnb/uh2-sebas/analysis/metagenomics/taxa_concordance/bracken_renorm_ILO_phyloseq.R1.K3.10.09.2024.csv'

#bracken han data
bracken_xu_source_file = '/restricted/projectnb/uh2-sebas/data/metagenomics/external_cohorts/Han_Chinese_Centenarian/processed_data/data_library/bracken_renorm_chinese_phyloseq.R1.K3.10.22.2024.rds'
bracken_xu_file = '/restricted/projectnb/uh2-sebas/analysis/metagenomics/taxa_concordance/bracken_renorm_xu_phyloseq.R1.K3.10.22.2024.csv'

#kraken2 database inspect file
k2_db_file = '/restricted/projectnb/uh2-sebas/data/metagenomics/Kraken2-DB/Kraken2Uniq-DB-09222023/inspect_db.sh.o2952984'


In [3]:
#OLD DATA LOADING CODE

#load metaphlan merged relative abundance tables for ILO and Han Chinese cohort
#table from phyloseq object
#mph4_ilo = pd.read_csv('/restricted/projectnb/uh2-sebas/analysis/metagenomics/taxa_concordance/ilo_mph4_04022024.csv')
#mph4_han = pd.read_table('/restricted/projectnb/uh2-sebas/analysis/humann_han_chinese/humann3.9/merged_out/raw_merged/merged_metaphlan4_bugs_list.tsv', skiprows=1)

#load kraken2uniq merged relative abundance tables for ILO and Han Chinese cohort
#k2u_ilo = pd.read_csv('/restricted/projectnb/uh2-sebas/analysis/metagenomics/taxa_concordance/bracken_count_ILO_phyloseq.filtered.06.11.2024.csv')
#k2u_han = pd.read_csv('/restricted/projectnb/uh2-sebas/analysis/metagenomics/taxa_concordance/bracken_count_chinese_phyloseq.filtered.05.14.2024.csv')

### Turn raw data into consistent dictionary structure for each data source
#### Nested dictionary structured with taxonomy levels as first layer of keys and ID's as second layer with names as values

In [3]:
#mapping length of id and name to correct taxonomic level
level_name_dict = {1: 'Kingdom', 2: 'Phylum', 3:"Class", 4:'Order', 5:'Family', 6:'Genus', 7:'Species'}

In [4]:
#process metaphlan database into dictionary of taxa ID's/names
mph_db_dict = {'Kingdom':{}, 'Phylum':{}, 'Class':{}, 'Order':{}, 'Family':{}, 'Genus':{}, 'Species':{}}
mph_db = pickle.load(bz2.open(mph_db_file,'rb'))
mph_db.pop('markers')
#loop through all species genome bins first
for name in mph_db['taxonomy'].keys():
    ncbi = mph_db['taxonomy'][name][0]
    # check each taxonomic level
    for level in range(7):
        sub_name = '|'.join(name.split('|')[:level+1])
        sub_ncbi = '|'.join(ncbi.split('|')[:level+1])
        #exclude taxa with incomplete classification
        if sub_ncbi.endswith('|') or '||' in sub_ncbi:
            break
        #add to dictionary
        if sub_ncbi not in mph_db_dict[level_name_dict[level+1]]:
            mph_db_dict[level_name_dict[level+1]][sub_ncbi] = sub_name


#add names/ID's for SGB's that collapse multiple species
#additional_taxa = list(mph4_db['merged_taxon'].values())
#add_taxa = [item for sublist in additional_taxa for item in sublist]
#for taxa_pair in add_taxa:
#    ncbi = taxa_pair[1]
#    name = taxa_pair[0]
    # check each taxonomic level
#    for level in range(7):
#        sub_name = '|'.join(name.split('|')[:level+1])
#        sub_ncbi = '|'.join(ncbi.split('|')[:level+1])
        #exclude taxa with incomplete classification
#        if sub_ncbi.endswith('|') or '||' in sub_ncbi:
#            break
        #add to dictionary
#        if sub_ncbi not in mph4_db_dict[level_name_dict[level+1]]:
#            mph4_db_dict[level_name_dict[level+1]][sub_ncbi] = sub_name


len(mph_db_dict['Species'])

20789

In [5]:
#process kraken database into dictionary of taxa ID's/names
kraken_db_dict = {'Kingdom':{}, 'Phylum':{}, 'Class':{}, 'Order':{}, 'Family':{}, 'Genus':{}, 'Species':{}}
kraken2uniq_db = pd.read_table(k2_db_file ,header=None, names=['percent_of_database_minimizers_in_clade', 'minimizers_in_clade', 'taxa_minimizers', 'taxa_level', 'taxa_id', 'taxa_name'] )
kraken_db_name_id = kraken2uniq_db[['taxa_level', 'taxa_id', 'taxa_name']].copy()

#add each new taxa represented to a large list, along with id
taxa_id_pairs = []
level_map = {'D':0, 'P':1, 'C':2, 'O':3, 'F':4, 'G':5, 'S':6}
taxonomy_id = ['','','','','','','']
taxonomy_name = ['','','','','','','']
for i in range(len(kraken_db_name_id)):
    level = kraken_db_name_id.loc[i,'taxa_level']
    if level == 'D':
        text_level = 'K'
    else:
        text_level = level
    num = str(kraken_db_name_id.loc[i,'taxa_id'])
    name = str(text_level.lower() + '__' + kraken_db_name_id.loc[i,'taxa_name'].strip())
    if level in level_map.keys():
        taxonomy_name[level_map[level]] = name
        taxonomy_id[level_map[level]] = num
        
        #wash out taxonomy and id labels after this newly updated one
        j = level_map[level]
        for m in range(len(taxonomy_name)):
            if m > j:
                taxonomy_name[m] = ''
                taxonomy_id[m] = ''
        
        #save completed taxa as a id/name pair
        taxa_id_pairs.append(('|'.join(taxonomy_name), '|'.join(taxonomy_id)))

for pair in taxa_id_pairs:
    name = pair[0]
    ncbi = pair[1]
    # check each taxonomic level
    for level in range(len(ncbi.split('|'))):
        sub_name = '|'.join(name.split('|')[:level+1])
        sub_ncbi = '|'.join(ncbi.split('|')[:level+1])
        #exclude taxa with incomplete classification
        if sub_ncbi.endswith('|') or '||' in sub_ncbi:
            break
        #add to dictionary
        if sub_ncbi not in kraken_db_dict[level_name_dict[level+1]]:
            kraken_db_dict[level_name_dict[level+1]][sub_ncbi] = sub_name
len(kraken_db_dict['Species'])        

23127

In [6]:
def process_taxa_csv_output(file, db_dict):
    data_dict = {'Kingdom':{}, 'Phylum':{}, 'Class':{}, 'Order':{}, 'Family':{}, 'Genus':{}, 'Species':{}}

    #read in source data, concatenate name and save species ID's
    df = pd.read_csv(file, index_col=0)
    names = ['|'.join(df.iloc[i].values[:]) for i in range(len(df))]
    sp_id = [str(i) for i in df.index]

    #match species ID with database entry to get full taxonomy ID
    ncbi_ids = []
    db_sp = list(db_dict['Species'].keys())
    for sp in sp_id:
        found = False
        for s in db_sp:
            if s.split('|')[-1] == sp:
                ncbi_ids.append(s)
                found = True
        if not found:
            print('Species not found with ID: ' + sp)
            ncbi_ids.append('')

    for name, ncbi in zip(names, ncbi_ids):
        # check each taxonomic level
        for level in range(len(ncbi.split('|'))):
            sub_name = '|'.join(name.split('|')[:level+1])
            sub_ncbi = '|'.join(ncbi.split('|')[:level+1])
            #exclude taxa with incomplete classification
            if sub_ncbi.endswith('|') or '||' in sub_ncbi:
                break
            #add to dictionary
            if sub_ncbi not in data_dict[level_name_dict[level+1]]:
                data_dict[level_name_dict[level+1]][sub_ncbi] = sub_name
    
    return data_dict

In [7]:
mph_ilo_dict = process_taxa_csv_output(mph_ilo_file, mph_db_dict)
bracken_ilo_dict = process_taxa_csv_output(bracken_ilo_file, kraken_db_dict)
mph_xu_dict = process_taxa_csv_output(mph_xu_file, mph_db_dict)
bracken_xu_dict = process_taxa_csv_output(bracken_xu_file, kraken_db_dict)

In [8]:
print(len(mph_ilo_dict['Species']))
print(len(bracken_ilo_dict['Species']))
print(len(mph_xu_dict['Species']))
print(len(bracken_xu_dict['Species']))

787
1044
898
1504


### Quality Control
#### Check that Metaphlan taxa are in Metaphlan Database, same for Kraken

In [9]:
def check_matching_keys(dict1, dict2):
    missing_taxa = {}  # Dictionary to store missing taxa for each level
    all_taxa_match = True  # Variable to track if all taxa IDs match
    
    # Iterate over the keys of the first dictionary
    for level, taxa_dict in dict1.items():
        # Check if the level exists in the second dictionary
        if level in dict2:
            # Get the taxa IDs from the first dictionary
            dict1_taxa_ids = set(taxa_dict.keys())
            # Get the taxa IDs from the second dictionary
            dict2_taxa_ids = set(dict2[level].keys())
            # Check if all taxa IDs from the first dictionary exist in the second dictionary
            if not dict1_taxa_ids.issubset(dict2_taxa_ids):
                # Update all_taxa_match to False if any taxa IDs are missing
                all_taxa_match = False
                # Find the missing taxa
                missing_taxa[level] = dict1_taxa_ids.difference(dict2_taxa_ids)
        else:
            # Update all_taxa_match to False if the level is missing in the second dictionary
            all_taxa_match = False
            # Store all taxa IDs as missing if the level is missing in the second dictionary
            missing_taxa[level] = set(taxa_dict.keys())
    
    return all_taxa_match, missing_taxa

In [10]:
all_taxa_match, missing_taxa = check_matching_keys(mph_xu_dict, mph_db_dict)
all_taxa_match

True

In [11]:
all_taxa_match, missing_taxa = check_matching_keys(bracken_xu_dict, kraken_db_dict)
all_taxa_match

True

### Taxa Overlap
#### Between Kraken and Metaphlan databases, and cohorts at each taxonomic level. Organize into dictionaries grouping by taxa in common and unique to a cohort/profiler

In [12]:
def taxa_overlap(dict1_name, dict2_name, dict1, dict2, verbose=True):
    categorized_taxa = {}  # Nested dictionary to store categorized taxa
    taxonomic_levels = ['Kingdom', 'Phylum', 'Class', 'Order', 'Family', 'Genus', 'Species']
    num_inconsistencies = 0
    mph_mismatch = []
    krk_mismatch = []
    
    # Iterate over the keys of the dictionaries
    for level, taxa_dict1 in dict1.items():
        # Initialize dictionaries to store categorized taxa at each level
        categorized_taxa[level] = {'Overlap': {}, dict1_name: {}, dict2_name: {}}
        level_index = taxonomic_levels.index(level)
        
        # Get taxa IDs from both dictionaries at the specified level
        taxa_ids_dict1 = set(taxa_id.split('|')[level_index] for taxa_id in taxa_dict1.keys()) 
        
        if level in dict2:
            taxa_ids_dict2 = set(taxa_id.split('|')[level_index] for taxa_id in dict2[level].keys())  
        else:
            taxa_ids_dict2 = set()
        
        # Find overlapping taxa IDs
        overlapping_taxa_ids = taxa_ids_dict1.intersection(taxa_ids_dict2)  

        #Check ID mismatch at higher levels of classification -- still currently counted as a match at current level
        for match in overlapping_taxa_ids:  
            for taxa_id1 in taxa_dict1:
                if taxa_id1.split('|')[level_index] == match:
                    for taxa_id2 in dict2[level]:
                        if taxa_id2.split('|')[level_index] == match:
                            if not taxa_id1 == taxa_id2:
                                num_inconsistencies+=1
                                if verbose:
                                    print("\n WARNING: MISMATCH AT HIGHER LEVEL \nMetaphlan ID: "+str(taxa_id1)+"\nMetaphlan Name: "+str(dict1[level][taxa_id1])+"\nBracken ID: "+str(taxa_id2)+"\nBracken Name: "+str(dict2[level][taxa_id2]) )
                                    #current_name, current_taxid = get_current_ncbi_info(match)
                                    #print("Current NCBI name: "+current_name+"\nCurrent NCBI ID: " +current_taxid)

                                    
                            # Store overlapping taxa
                            categorized_taxa[level]['Overlap'][taxa_id1] = dict1[level][taxa_id1]
        
        # Find taxa IDs only in dictionary1
        taxa_ids_dict1_only = taxa_ids_dict1.difference(taxa_ids_dict2)
        for match in taxa_ids_dict1_only:  
            for taxa_id in taxa_dict1:
                if taxa_id.split('|')[level_index] == match:  
                    # Store taxa only in dictionary1
                    categorized_taxa[level][dict1_name][taxa_id] = taxa_dict1[taxa_id]

        # Find taxa IDs only in dictionary2
        taxa_ids_dict2_only = taxa_ids_dict2.difference(taxa_ids_dict1)
        if level in dict2:
            for match in taxa_ids_dict2_only:  
                for taxa_id in dict2[level]:
                    if taxa_id.split('|')[level_index] == match: 
                        # Store taxa only in dictionary2
                        categorized_taxa[level][dict2_name][taxa_id] = dict2[level][taxa_id]
        else:
            categorized_taxa[level][dict2_name] = {}

    return categorized_taxa 

In [13]:
def categorize_unique_taxa(mph_taxa, bracken_taxa, mph_db, kraken_db):
    
    #get overlap between sample data with both profilers
    overlap = taxa_overlap('mph', 'kraken', mph_taxa, bracken_taxa, verbose=False)

    #get overlap between sample data and opposite profiler database
    mph_kraken_db_overlap = taxa_overlap('mph', 'kraken_db', mph_taxa, kraken_db, verbose=False)
    bracken_mph_db_overlap = taxa_overlap('mph_db', 'bracken', mph_db, bracken_taxa, verbose=False)

    #set up dictionary for results
    unique_taxa_overlap = {}
    taxonomic_levels = ['Kingdom', 'Phylum', 'Class', 'Order', 'Family', 'Genus', 'Species']
    
    for level in taxonomic_levels:

        #for each level compare ilo_overlap unique lists, and separate the ones in db_overlap overlap lists 
        unique_taxa_overlap[level] = {"mph_only_not_overlap" : {},"mph_only_overlap" : {},
                           "kraken_only_not_overlap" : {}, "kraken_only_overlap": {}}

        # Compare mph_only of dict1 with overlap of dict2
        for id_, name in overlap[level]["mph"].items():
            if id_ in mph_kraken_db_overlap[level]["Overlap"]:
                unique_taxa_overlap[level]["mph_only_overlap"][id_] = name
            else:
                unique_taxa_overlap[level]["mph_only_not_overlap"][id_] = name

        # Compare kraken_only of dict1 with overlap of dict2
        for id_, name in overlap[level]["kraken"].items():
            if id_ in bracken_mph_db_overlap[level]["Overlap"]:
                unique_taxa_overlap[level]["kraken_only_overlap"][id_] = name
            else:
                unique_taxa_overlap[level]["kraken_only_not_overlap"][id_] = name

    return unique_taxa_overlap

In [14]:
def get_current_ncbi_info(taxa_id):
    '''From a ncbi ID at any taxonomic level, report back the name of all higher levels up to the level provided as well as the most up to date ncbi ID'''

    os.system(f"datasets summary taxonomy taxon {taxa_id} --as-json-lines | dataformat tsv taxonomy --template tax-summary > temp.tsv")
    raw_taxa_df = pd.read_table('temp.tsv')

    taxa_df = raw_taxa_df[['Superkingdom name','Superkingdom taxid', 'Phylum name', 'Phylum taxid', 'Class name', 'Class taxid','Order name', 'Order taxid',
                       'Family name', 'Family taxid', 'Genus name', 'Genus taxid', 'Species name', 'Species taxid']]
    print(raw_taxa_df.loc[0,:])

    #shorten df to taxonomic level of input taxid
    taxa_df = taxa_df.loc[0,:][:(taxa_df.loc[0,:]).last_valid_index()]

    name_df = taxa_df.filter(like='name')
    taxid_df = taxa_df.filter(like='taxid')
    
    taxid = '|'.join(taxid_df.fillna('').astype(str))
    name = '|'.join(name_df.fillna('').astype(str))
    
    return name, taxid

In [16]:
xu_overlap = taxa_overlap('mph', 'kraken', mph_xu_dict, bracken_xu_dict, verbose=False)
ilo_overlap = taxa_overlap('mph', 'kraken', mph_ilo_dict, bracken_ilo_dict, verbose=False)
db_overlap = taxa_overlap('mph', 'kraken', mph_db_dict, kraken_db_dict, verbose=False)

In [17]:
xu_unique_cat = categorize_unique_taxa(mph_xu_dict, bracken_xu_dict, mph_db_dict, kraken_db_dict)
ilo_unique_cat = categorize_unique_taxa(mph_ilo_dict, bracken_ilo_dict, mph_db_dict, kraken_db_dict)

In [18]:
kraken_overlap = taxa_overlap('ilo', 'xu', bracken_ilo_dict, bracken_xu_dict, verbose=False)
mph_overlap = taxa_overlap('ilo', 'xu', mph_ilo_dict, mph_xu_dict, verbose=False)

## Summarize Overlap Taxa Counts

Print out for each level of taxonomy the table that compares across profilers and cohorts

In [21]:
kraken_overlap_data_dict = {'Kingdom':{}, 'Phylum':{}, 'Class':{}, 'Order':{}, 'Family':{}, 'Genus':{}, 'Species':{}}
for level in kraken_overlap_data_dict.keys():
    kraken_overlap_data_dict[level] = kraken_overlap[level]['Overlap']
mph_overlap_data_dict = {'Kingdom':{}, 'Phylum':{}, 'Class':{}, 'Order':{}, 'Family':{}, 'Genus':{}, 'Species':{}}
for level in mph_overlap_data_dict.keys():
    mph_overlap_data_dict[level] = mph_overlap[level]['Overlap']
ilo_overlap_data_dict = {'Kingdom':{}, 'Phylum':{}, 'Class':{}, 'Order':{}, 'Family':{}, 'Genus':{}, 'Species':{}}
for level in ilo_overlap_data_dict.keys():
    ilo_overlap_data_dict[level] = ilo_overlap[level]['Overlap']
xu_overlap_data_dict = {'Kingdom':{}, 'Phylum':{}, 'Class':{}, 'Order':{}, 'Family':{}, 'Genus':{}, 'Species':{}}
for level in xu_overlap_data_dict.keys():
    xu_overlap_data_dict[level] = xu_overlap[level]['Overlap']

all_overlap_cohort = taxa_overlap('ilo', 'han', ilo_overlap_data_dict, xu_overlap_data_dict, verbose=False)
all_overlap_profiler = taxa_overlap('mph', 'kraken', mph_overlap_data_dict, kraken_overlap_data_dict, verbose=False)

In [23]:
overlap_df_dict = {}
for level in db_overlap.keys():
    #create dataframe
    df = pd.DataFrame(index=['Kraken2', 'MetaPhlAn4', 'Profiler Overlap'], columns= ['Reference DB', 'ILO', 'Xu et. al', 'Cohort Overlap'])
    df['Reference DB'] = [(len(db_overlap[level]['Overlap']) + len(db_overlap[level]['kraken'])), (len(db_overlap[level]['Overlap']) + len(db_overlap[level]['mph'])),len(db_overlap[level]['Overlap'])] 
    df['ILO'] = [(len(ilo_overlap[level]['Overlap']) + len(ilo_overlap[level]['kraken'])), (len(ilo_overlap[level]['Overlap']) + len(ilo_overlap[level]['mph'])),len(ilo_overlap[level]['Overlap'])]
    df['Xu et. al'] = [(len(xu_overlap[level]['Overlap']) + len(xu_overlap[level]['kraken'])), (len(xu_overlap[level]['Overlap']) + len(xu_overlap[level]['mph'])),len(xu_overlap[level]['Overlap'])]
    df['Cohort Overlap'] = [len(kraken_overlap[level]['Overlap']), len(mph_overlap[level]['Overlap']), len(all_overlap_cohort[level]['Overlap'])]
    overlap_df_dict[level] = df
    print(level + '\n')
    print(df)
    print('\n')

Kingdom

                  Reference DB  ILO  Xu et. al  Cohort Overlap
Kraken2                      4    3          3               3
MetaPhlAn4                   3    3          3               3
Profiler Overlap             3    3          3               3


Phylum

                  Reference DB  ILO  Xu et. al  Cohort Overlap
Kraken2                     90   20         25              20
MetaPhlAn4                  93   12         12              12
Profiler Overlap            61   11         11              11


Class

                  Reference DB  ILO  Xu et. al  Cohort Overlap
Kraken2                    178   36         40              29
MetaPhlAn4                 131   22         24              22
Profiler Overlap           106   18         20              18


Order

                  Reference DB  ILO  Xu et. al  Cohort Overlap
Kraken2                    342   58         73              48
MetaPhlAn4                 279   38         45              37
Profiler Overlap  